In [29]:
import urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

# Using partners data from kivatools.com
partners = pd.read_csv('partners.csv', header = 0,encoding='latin-1')
n = 0
session = requests.Session()

# Scrape social badges of field partners
partners['badges'] = ''
partners['Anti-Poverty'] = ''
partners['Vulnerable Group'] = ''
partners['Client Voice'] = ''
partners['Family and Community Empowerment'] = ''
partners['Entrepreneurial Support'] = ''
partners['Facilitation of Savings'] = ''
partners['Innovation'] = ''

for i, row in partners.iterrows():

    try:
        
        #Getting HTML Data
        partnerid = row['ID']
        url = 'https://www.kiva.org/about/where-kiva-works/partners/' + str(partnerid)
        # https://www.kiva.org/about/where-kiva-works/partners/9
        
        r = session.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')

        badges = str(soup.findAll('div', class_= 'spStrengthItem'))
        badges2 = "".join(badges)
        
        if 'antipoverty' in badges2:
            partners.at[i,'Anti-Poverty'] = int(1)
        else:
            partners.at[i,'Anti-Poverty'] = int(0)

        if 'vulnerablegroup' in badges2:
            partners.at[i,'Vulnerable Group'] = int(1)
        else:
            partners.at[i,'Vulnerable Group'] = int(0)

        if 'clientvoice' in badges2:
            partners.at[i,'Client Voice'] = int(1)
        else:
            partners.at[i,'Client Voice'] = int(0)

        if 'familyandcommunity' in badges2:
            partners.at[i,'Family and Community Empowerment'] = int(1)
        else:
            partners.at[i,'Family and Community Empowerment'] = int(0)

        if 'entrepreneurial' in badges2:
            partners.at[i,'Entrepreneurial Support'] = int(1)
        else:
            partners.at[i,'Entrepreneurial Support'] = int(0)

        if 'facilitatesavings' in badges2:
            partners.at[i,'Facilitation of Savings'] = int(1)
        else:
            partners.at[i,'Facilitation of Savings'] = int(0)

        if 'innovation' in badges2:
            partners.at[i,'Innovation'] = int(1)
        else:
            partners.at[i,'Innovation'] = int(0)

        partners.at[i,'badges'] = badges2
        
    except AttributeError:
        partners.at[i,'badges'] = ''
        continue
    except ValueError:
        partners.at[i,'badges'] = ''


# partners.to_csv('partners_parse.csv', header=True, index=None)

In [41]:
# Scrape average cost to borrowers
partners['Avg_cost_to_borrowers'] = ''

for i, row in partners.iterrows():

    try:
        
        partnerid = row['ID']
        
        # Get HTML
        url = 'https://www.kiva.org/about/where-kiva-works/partners/' + str(partnerid)
        # https://www.kiva.org/about/where-kiva-works/partners/9
        
        r = session.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')

        cost = soup.find('div', class_='info')
        cost_text = " ".join(cost.text.replace('\r',' ').replace('\n',' ').strip().split())
        p1 = re.compile(r'Average Cost to Borrower:\s*([0-9]*)[%] PY')
        m = p1.search(cost_text)


        partners.at[i,'Avg_cost_to_borrowers'] = float(m.group(1))/100
    
    # Error Handling
    except AttributeError:
        partners.at[i,'Avg_cost_to_borrowers'] = ''
        continue
        
    except ValueError:
        partners.at[i,'Avg_cost_to_borrowers'] = ''
        continue
    except:
        partners.at[i,'Avg_cost_to_borrowers'] = ''
        continue
    


partners.to_csv('partners_parse.csv', header=True, index=None)

In [5]:
# Test
partners.head(5)

,ID,Name,Link,Status,Rating,Delinquency Rate,Delinquency Rate Note,Default Rate,Default Rate Note,Charges fees and interest?,...,URL,badges,Anti-Poverty,Vulnerable Group,Client Voice,Family and Community Empowerment,Entrepreneurial Support,Facilitation of Savings,Innovation,Avg_cost_to_borrowers
0,9,KREDIT Microfinance Institution,http://kiva.org/partners/9,active,4.5,2.8153,0,0.2743,0,1,...,http://www.credit.com.kh,"[<div class=""spStrengthItem"">\n<span class=""sp...",0,0,1,0,0,1,0,
1,15,South Pacific Business Development (SPBD) - Samoa,http://kiva.org/partners/15,active,3.5,2.1860,0,0.3609,0,1,...,http://www.spbdmicrofinance.com/,"[<div class=""spStrengthItem"">\n<span class=""sp...",1,0,0,0,0,1,0,
2,23,Microbanco Confianca,http://kiva.org/partners/23,active,2.5,1.2204,0,0.3345,0,1,...,http://hluvuku-adsema.org.mz/eng/hluvuku-adsema/,"[<div class=""spStrengthItem"">\n<span class=""sp...",0,0,0,1,0,0,1,
